Inspired by https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path+"/scripts")

In [3]:
import pandas as pd
import pickle

with open("../data/all_reviews.pkl","rb") as f:
    reviews = pickle.load(f)
    

In [4]:
with open("../data/job_titles.txt","r") as f:
    common_tech_titles = [ l.replace("\n","").strip() for l in f ]

import string
import re
from DataPrep import cleanTitle

common_tech_titles = list(map(cleanTitle,common_tech_titles))

display(set(common_tech_titles))



{'application developer',
 'application support analyst',
 'applications engineer',
 'associate developer',
 'chief information officer',
 'chief technology officer',
 'cloud architect',
 'cloud consultant',
 'cloud product project manager',
 'cloud services developer',
 'cloud software network engineer',
 'cloud system administrator',
 'cloud system engineer',
 'computer information research scientist',
 'computer information systems manager',
 'computer network architect',
 'computer programmer',
 'computer systems analyst',
 'computer systems manager',
 'customer support administrator',
 'customer support specialist',
 'data center support specialist',
 'data quality manager',
 'database administrator',
 'desktop support manager',
 'desktop support specialist',
 'director technology',
 'front end developer',
 'help desk specialist',
 'help desk technician',
 'information security',
 'java developer',
 'management information systems director',
 'net developer',
 'network administrat

## N-Gram with Top N Cosine

In [5]:
# clean_job_titles = reviews.clean_job_title.tolist()
clean_job_titles_filtered = reviews.clean_job_title.value_counts()
clean_job_titles_filtered = clean_job_titles_filtered.loc[clean_job_titles_filtered > 1].index.tolist()

clean_job_titles_filtered.extend(common_tech_titles)

print(len(clean_job_titles_filtered))

106261


In [6]:
import re

def ngrams(string, n=3):
    #string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in {}:'.format(clean_job_titles_filtered[0]))
print( ngrams(clean_job_titles_filtered[0]) )

All 3-grams in sales associate:
['sal', 'ale', 'les', 'es ', 's a', ' as', 'ass', 'sso', 'soc', 'oci', 'cia', 'iat', 'ate']


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(clean_job_titles_filtered)

print(len(vectorizer.get_feature_names()))

9125


In [8]:
print(clean_job_titles_filtered[0])
print( tf_idf_matrix[0] )

sales associate
  (0, 1202)	0.26292540368052825
  (0, 3570)	0.28059718134382955
  (0, 1708)	0.21463563116965367
  (0, 5433)	0.28027856751841695
  (0, 6842)	0.28019328630429796
  (0, 6915)	0.2760705507206696
  (0, 1192)	0.2377285822615709
  (0, 60)	0.2563267698347386
  (0, 6606)	0.3282008167188559
  (0, 2629)	0.2861858988943331
  (0, 4374)	0.2964736880627908
  (0, 1044)	0.29332914743884886
  (0, 6652)	0.29497902408255716


In [9]:
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [10]:
# import time
# t1 = time.time()
# matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.7)
# t = time.time()-t1
# print("SELFTIMED:", t)

In [11]:
from sparse_dot_topn import awesome_cossim_topn
import time

t1 = time.time()

matches = awesome_cossim_topn(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.7, use_threads=True, n_jobs=11)

t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 38.93819189071655


In [12]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [13]:
matches_df = get_matches_df(matches, clean_job_titles_filtered, top=False)
# matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)

,left_side,right_side,similairity
290957,associate store,associate store,1.000000
73953,department manager key carrier,operations department manager key carrier,0.893524
518244,customer solutions executive,customer solution,0.756920
607648,service representative marketing sales,marketing customer service representative,0.761189
93309,stock inventory associate,stock inventory,0.856562
25914,deli assistant,deli associate,0.738400
205769,layout designer,layout design engineer,0.777435
575152,customer service shop assistant,customer service assistant,0.707915
382851,assistant service operations manager,service operations manager,0.852222
587306,maintenance worker seasonal,maintenance worker operator,0.711889


In [14]:
matches_df.sort_values(['similairity'], ascending=True).head(10)

,left_side,right_side,similairity
343202,social media team,social media manager,0.700001
48086,social media manager,social media team,0.700001
436003,design supervisor project manager,design engineer project manager,0.700001
272281,técnico telecomunicações pleno,técnico eletrônica telecomunicações,0.700001
399530,técnico eletrônica telecomunicações,técnico telecomunicações pleno,0.700001
419344,garden associate receiving,sales associate receiving associate,0.700002
394408,salesman merchandising specialist,merchandising sales,0.700002
330914,rental technician,fleet rental technician,0.700002
430405,fleet rental technician,rental technician,0.700002
334014,developer internship,net developer intern,0.700003


In [15]:
filtered_jobs = matches_df.loc[matches_df.left_side.isin(common_tech_titles),["left_side","right_side"]]
print(filtered_jobs.shape)

pd.set_option("display.max_colwidth",500)
display(filtered_jobs.groupby("left_side")["right_side"].apply(list).reset_index())
pd.reset_option("display.max_colwidth")


(1316, 2)


,left_side,right_side
0,application developer,"[application developer, application developer, application developer sap, application developer team, application developer associate, application developer analyst, application developer specialist, net application developer, rational application developer, application developer intern, application developer, application developer, application developer sap, application developer team, application developer associate, application developer analyst, application developer specialist, net appl..."
1,application support analyst,"[application support analyst, application support analyst, applications support analyst, technical application support analyst, clinical application support analyst, application support, business application support analyst, application production support analyst, production application support analyst, application technical support analyst, application support analyst, application support analyst, applications support analyst, technical application support analyst, clinical application supp..."
2,applications engineer,"[applications engineer, applications engineer, applications engineer, applications engineer sales engineering, applications engineer intern, application operations engineer, application sales engineer, application engineer, technical applications engineer, applications engineering intern, applications engineer, applications engineer, applications engineer, applications engineer sales engineering, applications engineer intern, application operations engineer, application sales engineer, appli..."
3,associate developer,"[associate developer, associate developer, developer associate, associate development, developer developer, developer, software associate developer, associate development manager, store developer, associate software developer, associate developer, associate developer, developer associate, associate development, developer developer, developer, software associate developer, associate development manager, store developer, associate software developer]"
4,chief information officer,"[chief information officer, chief information officer, chief information office, chief information officer cio, chief information security officer, information officer, chief information systems, chief information officer, chief information officer, chief information office, chief information officer cio, chief information security officer, information officer, chief information systems]"
...,...,...
65,technology systems administrator,"[technology systems administrator, technology systems administrator, technology system administrator, systems administrator technology consultant, technology specialist system administrator, systems administrator, systems administrator, technology administrator, tier systems administrator, system administrator technology, technology systems administrator, technology systems administrator, technology system administrator, systems administrator technology consultant, technology specialist syst..."
66,telecommunications specialist,"[telecommunications specialist, telecommunications specialist, telecommunication specialist, telecommunications operations specialist, communications specialist, telecommunications, telecommunications sales, technology telecommunications specialist, telecommunications specialist project manager, telecommunications manager, telecommunications specialist, telecommunications specialist, telecommunication specialist, telecommunications operations specialist, communications specialist, telecommun..."
67,web administrator,"[web administrator, web administrator, web administrator, lab administrator, web content administrator, web administrator, web administrator, web administrator, lab administrator, web content administrator, web administrator, web administrator, web administrator, lab administrator, web content administrator]"
68,web developer,"[web developer, web developer, web developer, web developer 

In [16]:
sum(reviews.clean_job_title.isin(filtered_jobs.right_side.tolist()))

42590

## TF-IDF with K-Means

In [17]:

clean_job_titles = reviews.clean_job_title.unique().tolist()
print(len(clean_job_titles))

506654


In [18]:
import re

def ngrams(string, n=3):
    #string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in {}:'.format(common_tech_titles[0]))
print( ngrams(common_tech_titles[0]) )

All 3-grams in cloud architect:
['clo', 'lou', 'oud', 'ud ', 'd a', ' ar', 'arc', 'rch', 'chi', 'hit', 'ite', 'tec', 'ect']


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(common_tech_titles)

print(len(vectorizer.get_feature_names()))

340


In [20]:
from sklearn.neighbors import NearestNeighbors
import time

nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tf_idf_matrix)

In [21]:
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(clean_job_titles)
t = time.time()-t1
print("COMPLETED IN:", t)

getting nearest n...
COMPLETED IN: 8.940165042877197


In [22]:
print('finding matches...')
kmeans_matches = []
for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), common_tech_titles[j[0]],clean_job_titles[i]]
    kmeans_matches.append(temp)
print('Building data frame...')  
kmeans_matches = pd.DataFrame(kmeans_matches, columns=['Match confidence (lower is better)','left_side','right_side'])
print('Done')

finding matches...
Building data frame...
Done


In [23]:
idx = kmeans_matches["Match confidence (lower is better)"] < .4
kmeans_filtered_jobs = kmeans_matches.loc[idx,["left_side","right_side"]]
print(sum(idx))

pd.set_option("display.max_colwidth",700)
display( kmeans_filtered_jobs.groupby("left_side").right_side.apply(list).reset_index() )
pd.reset_option("display.max_colwidth")

1442


,left_side,right_side
0,application developer,"[application developer, sap abap application developer, mobile application developer, oracle application developer, application developer latam, application develope, application developer head, android application developer, ios application developer, mean stack application developer, application developer sql, android mobile application developer, ios mobile application developer, application developer mobile ios, websphere application developer, application developer j2ee, tririga application developer, application developer sap, application developer ios, application developer qlikview cognos, application developer oracle apps, maximo application developer, application developer lotu..."
1,application support analyst,"[application support analyst, business application support analyst, hcm application support analyst, peachtree accounting application support analyst, unix application support analyst, mainframe application support analyst, application support analyst oracle apps, application support analyst team, application support business analyst, level application support analyst, application support analysis qnxt, tax support analyst cash applications, linux application support analyst, business analyst application support, analyst application support, application support analyst offshore, level banking application support analyst, system application support analyst, application monitoring support ..."
2,applications engineer,"[applications engineer, delcam applications engineer, staff field applications engineer, field applications engineer, applications engineer hvcb, applications engineer cad cam, applications tailoring engineer, foundry applications engineer, applications engineer devops, applications engineer ic5, principle applications engineer, lighting applications engineer, oracle applications engineer, sas applications engineer, applications engineer cvp, sales applications engineer]"
3,associate developer,"[associate developer, associate developer dot net, associate oracle sql developer, associate etl developer, associate sql etl developer, associate sql developer, associate marklogic developer, associate full stack developer, associate developer full time]"
4,chief information officer,"[chief information officer, interim chief information officer, chief information office, chief information officer cio, chief information office cio, chief information officer west region, regional chief information officer]"
...,...,...
57,technology systems administrator,"[technology systems administrator, technology systemadministrator, nexgen technology systems administrator]"
58,telecommunications specialist,"[telecommunications specialist, engineering telecommunications specialist, telecommunications repair specialist, telecommunications account specialist, military telecommunications specialist, analyst telecommunications specialist, telecommunications systems specialist, telecommunications specialist csr]"
59,web administrator,"[web administrator, marketing web administrator, training advisor•web administrator, web hosting administrator]"
60,web developer,"[web developer, full stack web developer, web developer web developer, php web developer, ctl web developer, web developer oracle, web developer html5 developer, web developer jefe equipo, drupal web developer, angular web developer, mobile web developer, web developer fullstack, role web developer, global web developer, web developer sme, oracle sql web developer, j2ee web developer, staff web developer ebt, eftps web developer, sfdc web developer, web developer developer]"


In [24]:
sum(reviews.clean_job_title.isin(kmeans_filtered_jobs.right_side.tolist()))

36276

Merge both datsets

In [25]:

finalized_job_list = pd.concat([filtered_jobs, kmeans_filtered_jobs]).drop_duplicates()

finalized_job_list.shape

(1855, 2)

In [26]:
sum(reviews.clean_job_title.isin(finalized_job_list.right_side.tolist()))

45701

In [27]:
finalized_job_list.head()

finalized_job_list.to_csv("../data/finalized_job_list.csv", index=False)

Remove Jobs...

In [28]:
filter_titles = finalized_job_list.right_side.tolist()

exclude_keywords = [
    "camp"
    ,"tutor"
    ,"professor"
    ,"entry"
    ,"entries"
    ,"teacher"
    ,"audio"
    ,"civil"
    ,"facility"
    ,"facilities"
    ,"industrial"
    ,"metals"  
    ,"lab"
]

def excludeJobTItles(job):
    tokens = str(job).split(" ")
    return len( [ word for word in tokens if word in exclude_keywords ] ) == 0

filter_titles_2 = list( filter(excludeJobTItles, filter_titles) )

In [29]:
sum(reviews.clean_job_title.isin(filter_titles_2))

45687

In [30]:
import pandas as pd

pd.DataFrame(filter_titles_2,columns=["clean_job_title"]).to_csv("../data/filter_job_titles.csv",index=False)